**Import các thư viện cần thiết**

In [1]:
import os 
import random
from shutil import copyfile
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


**Định nghĩa các tham số**

In [2]:
# Define parameters
IMAGE_SIZE = 224
IMAGE_DATA = './data'

images = []
labels = []

**Hàm tiền xử lý ảnh, bao gồm**
* Resize image
* Normalization image
* Augumentation image
* Blancaed image

In [3]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

**Sinh dữ liệu mẫu phục vụ Trainning**

In [4]:
for path, subdirs, files in os.walk(IMAGE_DATA):
    for name in files:
        img_path = os.path.join(path, name)
        images.append(prepare_image(img_path))
        labels.append(path.split('/')[-1])
images = np.array(images)
# Remove single axis
images = np.squeeze(images, axis=1)

**Mapping label dữ liệu cho phù hợp mô hình boss -> 1, other -> 0**

In [5]:
mapped_labels = list(map(lambda x: 1 if x == 'data\\boss' else 0, labels))

from keras.utils import np_utils

y_data = np_utils.to_categorical(mapped_labels)

**Phân chia dữ liệu thành tập train, test theo tỉ lệ 0.3**

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, y_data, test_size=0.3)

** Định nghĩa mô hình mobilnet, cấu trúc mô hình như sau**

In [7]:
model = keras.applications.mobilenet.MobileNet(classes=2, weights=None)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseC

**Hàm mất mát cross entropy, optimizers adam, metric sử dụng là accuracy**

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Đặt check point**

In [9]:
from keras.callbacks import ModelCheckpoint

model_file = "model_mobilenet.hdf5"

checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

**Training**

In [10]:
model.fit(x=X_train, y=y_train, batch_size=8, epochs=5, verbose=1, validation_data=(X_test, y_test), callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 29 samples, validate on 13 samples
Epoch 1/5
29/29 [==============================] - 30s 1s/step - loss: 0.5952 - acc: 0.8276 - val_loss: 0.3759 - val_acc: 0.7692

Epoch 00001: val_loss improved from inf to 0.37585, saving model to model_mobilenet.hdf5
Epoch 2/5
29/29 [==============================] - 23s 808ms/step - loss: 0.0704 - acc: 1.0000 - val_loss: 0.1839 - val_acc: 1.0000

Epoch 00002: val_loss improved from 0.37585 to 0.18391, saving model to model_mobilenet.hdf5
Epoch 3/5
29/29 [==============================] - 24s 811ms/step - loss: 0.0675 - acc: 1.0000 - val_loss: 5.3711 - val_acc: 0.4615

Epoch 00003: val_loss did not improve from 0.18391
Epoch 4/5
29/29 [==============================] - 23s 798ms/step - loss: 0.0097 - acc: 1.0000 - val_loss: 8.4321 - val_acc: 0.4615

Epoch 00004: val_loss did not improve from 0.18391
Epoch 5/5
29/29 [

**Lưu lại mô hình**

In [11]:
model.save('model.h5')

In [14]:
x = prepare_image('./data/boss/WIN_20180521_19_34_18_Pro.jpg')
print(model.predict(x))

[[9.9999952e-01 4.3087027e-07]]
